In [54]:
from bs4 import BeautifulSoup
import bs4
import requests as req
import json, re

In [ ]:
### Get the links to every chapter of every book

# First, get a single page's content and scan through the table of contents for all book elements
r = req.get('http://www.perseus.tufts.edu/hopper/text?doc=Perseus%3Atext%3A1999.02.0137%3Abook%3D1%3Achapter%3Ddedication')
soup = BeautifulSoup(r.text)
table_of_contents = soup.find(id='side_toc')

table_of_contents_children = table_of_contents.contents

len(table_of_contents_children)

38

In [21]:
BASE_URL = 'http://www.perseus.tufts.edu/hopper/text'

In [ ]:
# Gets the links for every chapter within a given book
def parse_chapter_links(chapter: bs4.element.Tag):
    whitespace_regex = r'\s*\n\s*'
    book_title = re.sub(whitespace_regex, ' ', chapter.find_all(name='a')[1].span.text.strip())
    chapter_links = chapter.div.find_all(name='a')
    
    parsed_chapter_links = []
    
    for link in chapter_links:
        link_data = {'book_title': book_title}
        link_data['url'] = BASE_URL + link.attrs['href']
        link_data['chapter_name'] = re.sub(whitespace_regex, ' ', link.span.text.strip())
    
        parsed_chapter_links.append(link_data)
        
    return parsed_chapter_links

In [ ]:
# Get all the links from all the books
all_chapters = []
for child in table_of_contents_children[:-1]:
    all_chapters.extend(parse_chapter_links(child))
    
# ...and write out to a JSON for future reference
with open('out.json', 'w') as f:
    json.dump(all_chapters, f, indent=2, ensure_ascii=False)

In [ ]:
# TODO: parse throuh the text of each chapter and collect text, footnotes, and separate into paragraphs for RAG/fine-tuning